In [ ]:
import json

from pitchfilter.pitchfilter import PitchFilter
from tonicidentifier.toniclastnote import TonicLastNote
from morty.pitchdistribution import PitchDistribution
from notemodel.notemodel import NoteModel

from adaptivetuning.tuner.tuner import Tuner
from adaptivetuning.synthesizer.synth_symbtr import synth_symbtr

from matplotlib import pyplot as plt
%matplotlib inline
%pdb

In [ ]:
# loading pitch for adaptive tuning
pitch = json.load(open("sampledata/huseyni--sazsemaisi--aksaksemai----tatyos_efendi/8b8d697b-cad9-446e-ad19-5e85a36aa253.json", 'r'))['pitch']
makam = 'huseyni'

# Post process the pitch track to get rid of spurious pitch estimations and correct octave errors
pitch_filter = PitchFilter()
pitch = pitch_filter.run(pitch)

In [ ]:
# tonic identification for the related recording of SymbTr
tonic_identifier = TonicLastNote()
tonic = tnc.identify(pitch)[0]

In [ ]:
# compute the pitch distribution
pitch_distribution = PitchDistribution.from_hz_pitch(pitch[:,1], ref_freq=tonic['value'])

# obtain the tuning
model = NoteModel()
stablenotes = model.calculate_notes(pitch_distribution, tonic['value'], makam,
                                    min_peak_ratio=0.1)

# plot the tuning
model.plot(pitch_distribution, stablenotes)
plt.xlim([160, 450])
plt.show()

# Adapting the tuning and synthesizing the SymbTr

This part synthesizes the score according to the related audio recording.
Extracted stable pitches are used as reference in synth function.

**It takes a time w.r.t. the length of given SymbTr.**
If you want to obseve the status of the process, give *"verbose"* flag as *true*.

Possible synth types are *'sine'* and *'karplus'*
*'Sine'* option takes less time.

In [ ]:
adapt = Tuner()
theoretical_histogram, adapted_histogram = adapt.adapt_score_frequencies(musicxml_path="sampledata/huseyni--sazsemaisi--aksaksemai----tatyos_efendi/huseyni--sazsemaisi--aksaksemai----tatyos_efendi.xml",
                                                                         performed_tonic=tonic['value'],
                                                                         stable_pitches=stablenotes,
                                                                         type='karplus',
                                                                         verbose=False)

In [ ]:
stablenotes